In [11]:
import os
import time
import csv
import re # get_post_content 함수에서 사용
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import random # time.sleep()에 사용

# --- 1. 설정 ---
# DC_GALLERY_URL = "https://gall.dcinside.com/mgallery/board/lists/?id=platinalab"
DC_GALLERY_URL = "https://gall.dcinside.com/board/lists?id=baseball_new11"
BASE_DC_URL = "https://gall.dcinside.com" # 링크 생성 시 사용
TARGET_POST_COUNT = 300 # 수집할 목표 게시글 수
OUTPUT_CSV_FILE = "./data/crawled_posts_basic.csv" # 저장될 CSV 파일명 변경
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# --- 2. 웹 스크레이핑 함수 ---

def get_post_links_and_titles(gallery_url, target_count):
    """갤러리 목록 페이지에서 게시글 링크와 제목을 수집합니다."""
    post_details = []
    page = 1
    collected_count = 0
    # tqdm의 total을 target_count로 설정하여 진행률 표시
    pbar = tqdm(total=target_count, desc="게시글 링크 수집 중")

    while collected_count < target_count:
        try:
            current_url = f"{gallery_url}&page={page}"
            response = requests.get(current_url, headers=HEADERS, timeout=10)
            response.raise_for_status() # HTTP 오류 발생 시 예외 발생
            soup = BeautifulSoup(response.text, 'html.parser')

            # 일반 게시글 선택자 (공지 제외)
            posts_in_page = soup.select('table.gall_list tbody tr.ub-content.us-post')

            if not posts_in_page:
                if page == 1 and collected_count == 0:
                    print(f"페이지 {page}에서 게시글을 찾을 수 없습니다. CSS 선택자('table.gall_list tbody tr.ub-content.us-post')가 정확한지, "
                          "또는 웹사이트 구조가 변경되었는지 확인해주세요.")
                else:
                    print(f"페이지 {page}에서 더 이상 게시글을 찾을 수 없어 수집을 중단합니다.")
                break # 더 이상 게시글이 없으면 루프 종료

            for post_row in posts_in_page:
                # 제목 링크 선택 (공지 아이콘 클래스 제외)
                title_tag = post_row.select_one('td.gall_tit a:not([class*="icon_notice"])')

                if title_tag and title_tag.has_attr('href'):
                    title = title_tag.get_text(strip=True)
                    relative_link = title_tag['href']

                    if relative_link.startswith('/'):
                        link = BASE_DC_URL + relative_link
                    else:
                        link = relative_link
                    
                    # 중복 링크 방지 (이미 수집된 링크인지 확인)
                    if not any(d['link'] == link for d in post_details):
                        post_details.append({'title': title, 'link': link})
                        collected_count += 1
                        pbar.update(1) # 진행 바 업데이트

                    if collected_count >= target_count:
                        break # 목표 개수 도달 시 내부 루프 종료
            
            if collected_count >= target_count:
                break # 목표 개수 도달 시 외부 루프 종료

            page += 1
            time.sleep(random.uniform(0.5, 1.5)) # 서버 부하 감소를 위한 딜레이

        except requests.exceptions.Timeout:
            print(f"오류: 페이지 {page} 요청 시간 초과. 다음 페이지로 넘어갑니다.")
            page += 1
            time.sleep(1)
            continue
        except requests.exceptions.RequestException as e:
            print(f"오류: 페이지 {page} 스크레이핑 중 요청 오류 발생 - {e}")
            break 
        except Exception as e:
            print(f"알 수 없는 오류 발생 (페이지 {page} 처리 중): {e}")
            break
            
    pbar.close() # 진행 바 닫기
    if not post_details and target_count > 0:
        print("수집된 게시글이 전혀 없습니다. 프로그램 초기의 CSS 선택자, 네트워크 연결, 또는 대상 웹사이트의 접근성을 확인해주세요.")
    return post_details[:target_count]

def get_post_content(post_url):
    """개별 게시글 페이지에서 본문 내용을 가져옵니다."""
    try:
        response = requests.get(post_url, headers=HEADERS, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        
        content_div = soup.select_one('div.writing_view_box')
        
        if content_div:
            # 불필요한 태그 제거
            for unwanted_tag in content_div.find_all(['script', 'style', 'iframe', 'ins', 'figure', 
                                                     '.adv_bottom_title', 'div.btn_recommend_box', 
                                                     'div.json_dccon_viewer', 'div.social_area',
                                                     'div.gallery_info_bottom', 'div.related_cont']):
                unwanted_tag.decompose()
            
            text_parts = []
            for element in content_div.descendants:
                if isinstance(element, str): # NavigableString
                    stripped_text = element.strip()
                    if stripped_text:
                        text_parts.append(stripped_text)
                elif element.name == 'br':
                    if text_parts and text_parts[-1] != '\n' and text_parts[-1].strip() != '':
                         text_parts.append('\n')

            content = " ".join(text_parts)
            content = re.sub(r'\s+', ' ', content).strip()
            content = content.replace(' \n ', '\n').replace('\n ', '\n').replace(' \n', '\n')

            # content_excerpt를 위해 전체 내용을 반환 (길이 제한은 메인 로직에서)
            return content
        else:
            return "본문 내용을 찾을 수 없습니다."
            
    except requests.exceptions.Timeout:
        return f"게시글 내용 로드 실패 (타임아웃): {post_url}"
    except requests.exceptions.RequestException as e:
        return f"게시글 내용 로드 실패: {e}"
    except Exception as e:
        return f"알 수 없는 오류로 내용 로드 실패: {e}"

# --- 3. 메인 로직 ---
print(f"DCInside 게시글 크롤링을 시작합니다.")

# 데이터 저장 디렉토리 생성
if not os.path.exists('./data'):
    os.makedirs('./data')
    print(" './data' 디렉토리가 생성되었습니다.")

print(f"'{DC_GALLERY_URL}'에서 최대 {TARGET_POST_COUNT}개의 게시글 링크를 수집합니다.")
posts_to_scrape = get_post_links_and_titles(DC_GALLERY_URL, TARGET_POST_COUNT)

if not posts_to_scrape:
    print("수집할 게시글이 없습니다. 프로그램을 종료합니다.")
    exit() # return 대신 exit() 사용 (함수 외부이므로)

print(f"\n총 {len(posts_to_scrape)}개의 게시글 링크 수집 완료.")
all_scraped_data = []

print("\n게시글 내용 스크레이핑 중...")
for post_detail in tqdm(posts_to_scrape, desc="게시글 내용 수집 중"):
    post_title = post_detail['title']
    post_link = post_detail['link']
    
    post_content_full = get_post_content(post_link)

    content_excerpt = "내용 없음"
    if post_content_full and not post_content_full.startswith("본문 내용을 찾을 수 없습니다.") and not post_content_full.startswith("게시글 내용 로드 실패"):
        # 본문 미리보기는 200자로 제한하고, 줄바꿈은 공백으로 대체
        content_excerpt = post_content_full[:200].replace('\n', ' ') + "..."
    elif post_content_full.startswith("본문 내용을 찾을 수 없습니다."):
        content_excerpt = "본문 내용을 찾을 수 없습니다."
    elif post_content_full.startswith("게시글 내용 로드 실패"):
        content_excerpt = post_content_full # 오류 메시지 그대로 사용

    all_scraped_data.append({
        "title": post_title,
        "link": post_link,
        "content_excerpt": content_excerpt
    })
    
    # 각 게시물 처리 후 짧은 휴식 (서버 부하 감소)
    time.sleep(random.uniform(0.3, 0.8)) # 딜레이 약간 줄임

# --- 4. 결과를 CSV 파일로 저장 ---
print(f"\n수집된 데이터를 '{OUTPUT_CSV_FILE}' 파일로 저장합니다.")
if not all_scraped_data:
    print("저장할 데이터가 없습니다.")
else:
    try:
        # CSV 파일 저장 디렉토리 확인 및 생성
        output_dir = os.path.dirname(OUTPUT_CSV_FILE)
        if output_dir and not os.path.exists(output_dir): # output_dir이 빈 문자열이 아니고, 존재하지 않으면
            os.makedirs(output_dir)
            print(f"저장 디렉토리 '{output_dir}'가 생성되었습니다.")

        with open(OUTPUT_CSV_FILE, mode='w', newline='', encoding='utf-8-sig') as file:
            # 저장할 필드명 정의
            writer = csv.DictWriter(file, fieldnames=["title", "link", "content_excerpt"])
            writer.writeheader() # 헤더 작성
            writer.writerows(all_scraped_data) # 모든 데이터 한번에 작성
            
        print(f"'{OUTPUT_CSV_FILE}' 파일 저장 완료. 총 {len(all_scraped_data)}개 게시글 정보 저장.")
    except IOError as e:
        print(f"오류: '{OUTPUT_CSV_FILE}' 파일 저장에 실패했습니다. - {e}")
    except Exception as e:
        print(f"CSV 저장 중 알 수 없는 오류 발생: {e}")

print("\n프로그램이 종료되었습니다.")

DCInside 게시글 크롤링을 시작합니다.
'https://gall.dcinside.com/board/lists?id=baseball_new11'에서 최대 300개의 게시글 링크를 수집합니다.


게시글 링크 수집 중: 100%|██████████| 300/300 [00:06<00:00, 46.43it/s]



총 300개의 게시글 링크 수집 완료.

게시글 내용 스크레이핑 중...


게시글 내용 수집 중: 100%|██████████| 300/300 [03:12<00:00,  1.56it/s]


수집된 데이터를 './data/crawled_posts_basic.csv' 파일로 저장합니다.
'./data/crawled_posts_basic.csv' 파일 저장 완료. 총 300개 게시글 정보 저장.

프로그램이 종료되었습니다.
